# Merchant Loan Valuation Portfolio

In [1]:
#import necessary libraries
import numpy as np
import pandas as pd

In [2]:
# Import the data and store it in a variable.
data = pd.read_csv(r"C:\Users\ADMIN\Downloads\Data.csv",sep=';')

In [3]:
data.head()

,Unnamed: 0,Origination Amount,31.05.2019,30.06.2019,31.07.2019,31.08.2019,30.09.2019,31.10.2019,30.11.2019,31.12.2019,...,31.03.2020,30.04.2020,31.05.2020,30.06.2020,31.07.2020,31.08.2020,30.09.2020,31.10.2020,30.11.2020,31.12.2020
0,31.05.2019,10018746.17,1443069.08,3332200.33,1328138.75,928085.74,736418.27,539403.31,427557.86,324459.32,...,116684.68,92699.67,63399.66,53265.12,37121.13,29787.10,24524.90,18085.94,16581.01,11442.97
1,30.06.2019,10868379.04,0.00,1392751.60,3011884.91,1237868.70,970929.28,892351.83,668767.02,505612.59,...,255222.42,198833.96,161996.73,138461.91,92346.68,79641.30,63457.44,52373.85,43374.70,37404.87
2,31.07.2019,10733932.61,0.00,0.00,1537650.24,2953335.55,1208316.08,879375.19,711016.84,658251.40,...,302575.54,258652.52,191798.05,170027.54,127574.33,110301.21,89766.69,64746.84,61408.92,50312.70
3,31.08.2019,12558727.02,0.00,0.00,0.00,1617681.94,4082016.00,1387474.94,1247623.59,886293.35,...,417223.56,336686.08,253556.20,200066.59,151859.74,109973.00,90228.14,70661.50,53102.83,47069.84
4,30.09.2019,14505071.44,0.00,0.00,0.00,0.00,1992242.84,3930445.60,1394620.78,1227905.58,...,628429.48,589692.85,457299.31,323764.87,288152.28,239872.99,192246.98,171550.69,142575.97,116853.05


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          20 non-null     object 
 1   Origination Amount  20 non-null     float64
 2   31.05.2019          20 non-null     float64
 3   30.06.2019          20 non-null     float64
 4   31.07.2019          20 non-null     float64
 5   31.08.2019          20 non-null     float64
 6   30.09.2019          20 non-null     float64
 7   31.10.2019          20 non-null     float64
 8   30.11.2019          20 non-null     float64
 9   31.12.2019          20 non-null     float64
 10  31.01.2020          20 non-null     float64
 11  29.02.2020          20 non-null     float64
 12  31.03.2020          20 non-null     float64
 13  30.04.2020          20 non-null     float64
 14  31.05.2020          20 non-null     float64
 15  30.06.2020          20 non-null     float64
 16  31.07.2020

In [5]:
#Parameters
n_forecast = 30  # number of forecast periods
n_vintage = len(data) # number of vintages(rows in data)
discount_rate = 0.025 # annual discount rate


In [6]:
#Extract historical cash flows (as a matrix) and originated amounts

historical_cf = data.iloc[:, 2:].values  # Matrix of cash flows already observed
amount_originated = data.iloc[:, 1].values  # Vector of amounts originated per vintage


In [7]:
# Number of periods remaining per vintage
periods_remaining = n_forecast - np.arange(1, n_vintage + 1)


In [8]:
# Calculate repayment percentages (historical payments as a percentage of the originated amount per vintage)
paid_percentages = historical_cf / amount_originated[:, np.newaxis]


In [9]:
# Repayment percentages for the first and second periods
first_period = np.diag(paid_percentages)  # First repayment percentage (diagonal)
second_period = np.concatenate(
    [np.diag(paid_percentages[:-1, 1:]), [paid_percentages[-1, -1] * 2]]
)  # Second repayment percentage (adjust for the last vintage)


In [10]:
# Initialize the matrix of repayment percentages
p = np.zeros((n_vintage, n_forecast))

# Assign first and second periods to the repayment matrix
p[:, 0] = first_period
p[:, 1] = second_period

In [11]:
#Calculate expected repayment percentages for the forecast periods
for i in range(n_vintage):
    for j in range(2, n_forecast):
        p[i, j] = max(0, p[i, 1] * np.log(1 + (1 - np.sum(p[i, :j]))) * (1 - (j) / n_forecast))


In [12]:
# Initialize the forecast matrix
p_forecast = np.zeros((n_vintage, n_forecast - 1))


In [13]:
# Assign first and second periods to the repayment matrix
p[:, 0] = first_period
p[:, 1] = second_period


In [14]:
#Calculate expected repayment percentages for the forecast periods
for i in range(n_vintage):
    for j in range(2, n_forecast):
        p[i, j] = max(0, p[i, 1] * np.log(1 + (1 - np.sum(p[i, :j]))) * (1 - (j) / n_forecast))


In [15]:
# Initialize the forecast matrix
p_forecast = np.zeros((n_vintage, n_forecast - 1))


In [16]:
# Assign forecasted repayment percentages to the correct periods
for i in range(n_vintage):
    for j in range(int(periods_remaining[i])):
        p_forecast[i, j] = p[i, n_forecast - int(periods_remaining[i]) + j]


In [17]:
# Calculate the discount factors (monthly)
discount_factors = 1 / (1 + discount_rate) ** (np.arange(1, n_forecast) / 12)


In [18]:
# Calculate the present value of the forecasted cash flows
pv = (p_forecast * discount_factors) * amount_originated[:, np.newaxis]


In [19]:
# Calculate the total present value (portfolio value)
result = np.sum(pv)

In [20]:
#Print the result
print(f"The fair value estimate for the portfolio is {round(result, 2)} Swiss Francs")

The fair value estimate for the portfolio is 42792456.39 Swiss Francs
